###### Creates heatmaps using the gmaps API
heatmap is created for different weather parameter from the dataframe

In [1]:
#imports the dependencies
import pandas as pd
import gmaps
import requests

#imports the api key
from config import g_key

In [2]:
#stores the csv saved and created into a dataframe
city_data_df = pd.read_csv('weather_data/cities.csv')
#city_data_df.head()

In [3]:
#checking for data types
#gmaps require either an integer or floating-point decimal number

city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
#configures gmaps to use your google API key
gmaps.configure(api_key = g_key)

#removes the negative weight since it cant hold negative values
#gets the max temp
max_temp = city_data_df['Max Temp']

temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
#creating the heat map

#assigns the locations to an array of latitude and longitude pairs
locations = city_data_df[['Lat', 'Lng']]

#gets the max temp
max_temp = city_data_df['Max Temp']

#assigns the figure variable
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

#assigns the heatmap variable
#previous for loop replaced with list comprehension
heat_layer = gmaps.heatmap_layer(locations, weights = [max(temp, 0) for temp in max_temp], 
                                 dissipating = False, max_intensity = 300, point_radius = 4)

#adds the heatmap layer
fig.add_layer(heat_layer)

#calls the figure to plot data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#heatmap of percent humidity

#assigns location to an array of latitude and longitude pairs
locations = city_data_df[['Lat', 'Lng']]

#gets the humidity values from the dataframe
humidity = city_data_df['Humidity']

#assigns the figure variable
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

#assigns the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                             dissipating = False, max_intensity = 300,
                             point_radius = 4)

#adds the heatmap layer
fig.add_layer(heat_layer)

#calls the fig
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#heatmap of percent cloudiness

#assigns location to an array of latitude and longitude pairs
locations = city_data_df[['Lat', 'Lng']]

#gets the Cloudiness values from the dataframe
clouds = city_data_df['Cloudiness']

#assigns the fig variable
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

#assigns the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights = clouds,
                                dissipating = False, max_intensity = 300,
                                point_radius = 4)

#adds the heatmap layer to the figure
fig.add_layer(heat_layer)

#calls the fig
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#heatmap of the wind speeds

#assigns the location to an array of latitudes and longitudes pairs
locations = city_data_df[['Lat', 'Lng']]

#gets the wind speed values from the dataframe
wind = city_data_df['Wind Speed']

#assigns the figure variable
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

#assigns the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights = wind,
                                dissipating = False, max_intensity = 300,
                                point_radius = 4)

#adds the heatmap layer to the figure
fig.add_layer(heat_layer)

#calls the figure
fig

Figure(layout=FigureLayout(height='420px'))

###### Gets an input for vacation criteria
the user provides a max and min temp so that a location search can be performed and thus reccomendations

In [9]:
#asks for user input to add a max and min temps
min_temp = float(input('What is the minimum temperature (F) you would like for your vacation destination? '))
max_temp = float(input('What is the maximum temperature (F) you would like for your vacation destination? '))

What is the minimum temperature (F) you would like for your vacation destination? 75
What is the maximum temperature (F) you would like for your vacation destination? 90


In [10]:
#filters the dataset to find cities within the user input's parameters and drops any with null values
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)].dropna()

#preferred_cities_df.head()

In [11]:
#checked any cities for null values
#dropna() method was used when filtering for the new dataframe as well
preferred_cities_df.count()

City_ID       163
City          163
Country       163
Date          163
Lat           163
Lng           163
Max Temp      163
Humidity      163
Cloudiness    163
Wind Speed    163
dtype: int64

###### maps the vacation critera
maps the vacation criteria based on a new filtered dataframe from the user input's parameters

In [12]:
#copies some columns from the preferred_cities_df and adds a new column called 'Hotel Name'
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''
#hotel_df.head()

C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
#retrieves the hotels from a nearby search using Google Places API

#sets the parameters to search for a hotel
params = {'radius': 5000,
         'type': 'lodging',
         'key': g_key}

#iterates through the dataframe
for index, row in hotel_df.iterrows():
    
    #gets the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    
    #adds the latitude and longitude to the location key for the params dictionary
    params['location'] = f'{lat},{lng}'
    
    #uses the search term: 'lodging' and our latitude and longitude
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    #makes the request and gets the JSON data from the search
    hotels = requests.get(base_url, params = params).json()
    
    try:
        #grabs the first hotel from the results and stores the name
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    
    except (IndexError):
        print('Hotel not found... skipping')

C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


KeyboardInterrupt: 

In [14]:
#checks if hotel names are added
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Amahai,ID,79.02,-3.3333,128.9167,HOTEL ISABELA
5,Avarua,CK,82.40,-21.2078,-159.7750,Paradise Inn
6,Trinidad,BO,87.80,-14.8333,-64.9000,Hotel Piesta
18,Gao,ML,78.21,16.6362,1.6370,
21,Souillac,MU,80.60,-20.5167,57.5167,Shanti Maurice Resort & Spa


In [18]:
#heatmap of of max temps from the hotel dataframe
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#store the dataframe row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

#assigns the location to an array of latitudes and longitudes pairs
locations = hotel_df[['Lat', 'Lng']]

#gets the wind speed values from the dataframe
max_temp = hotel_df['Max Temp']

#assigns the figure variable
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

#assigns the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp,
                                dissipating = False, max_intensity = 300,
                                point_radius = 4)

#assigns the markers variable
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

#adds the marker layer to the figure
fig.add_layer(marker_layer)

#adds the heatmap layer to the figure
fig.add_layer(heat_layer)

#calls the figure
fig

Figure(layout=FigureLayout(height='420px'))